In [3]:
import numpy as np
import random

# Datos del problema
costos_transporte = np.array([[1, 4, 3, 6],
                              [4, 1, 4, 5],
                              [3, 4, 1, 4],
                              [6, 5, 4, 1]])

costos_generacion = np.array([680, 720, 660, 750])

# Necesidades de energía de las ciudades (en GW)
necesidades = np.array([4, 3, 5, 3])

# Capacidades de las plantas de generación (en GW)
capacidades = np.array([3, 6, 5, 4])

# Parámetros del algoritmo genético
num_individuos = 100
num_generaciones = 500
probabilidad_cruce = 0.7
probabilidad_mutacion = 0.1
tamaño_torneo = 5

# Función de aptitud (costo total)
def calcular_aptitud(individuo):
    # Convertir el individuo en una matriz de despacho de energía (4x4)
    despacho = individuo.reshape(4, 4)
    
    # Costo de transporte
    costo_transporte = np.sum(despacho * costos_transporte)
    
    # Costo de generación
    costo_generacion = np.sum(despacho.sum(axis=1) * costos_generacion)
    
    # Verificar las restricciones de capacidad y necesidades
    if any(despacho.sum(axis=1) > capacidades) or any(despacho.sum(axis=0) < necesidades):
        return float('inf')  # Si hay una violación de las restricciones, devolver un costo muy alto
    
    # Sumar los costos de transporte y generación
    return costo_transporte + costo_generacion

# Inicialización de la población
def inicializar_poblacion():
    poblacion = []
    while len(poblacion) < num_individuos:
        # Crear un individuo aleatorio respetando las restricciones
        despacho = np.zeros((4, 4))
        for i in range(4):
            for j in range(4):
                # Generar una cantidad de energía que no exceda la capacidad de la planta ni la necesidad de la ciudad
                despacho[i, j] = random.uniform(0, min(capacidades[i], necesidades[j]))
        
        # Verificar si la solución cumple las restricciones
        if np.all(despacho.sum(axis=1) <= capacidades) and np.all(despacho.sum(axis=0) >= necesidades):
            poblacion.append(despacho.flatten())
    
    return np.array(poblacion)

# Selección por torneo
def seleccion(poblacion):
    seleccionados = []
    for _ in range(num_individuos):
        torneo = random.sample(range(num_individuos), tamaño_torneo)
        mejor_individuo = min(torneo, key=lambda idx: calcular_aptitud(poblacion[idx]))
        seleccionados.append(poblacion[mejor_individuo])
    return np.array(seleccionados)

# Cruce (crossover)
def cruce(padre1, padre2):
    punto_cruce = random.randint(1, len(padre1) - 1)
    hijo1 = np.concatenate((padre1[:punto_cruce], padre2[punto_cruce:]))
    hijo2 = np.concatenate((padre2[:punto_cruce], padre1[punto_cruce:]))
    return hijo1, hijo2

# Mutación
def mutacion(individuo):
    if random.random() < probabilidad_mutacion:
        idx = random.randint(0, len(individuo) - 1)
        individuo[idx] = random.uniform(0, min(capacidades[idx // 4], necesidades[idx % 4]))
    return individuo

# Algoritmo genético
def algoritmo_genetico():
    poblacion = inicializar_poblacion()
    
    for generacion in range(num_generaciones):
        # Evaluar la población
        aptitudes = np.array([calcular_aptitud(individuo) for individuo in poblacion])
        
        # Mostrar el mejor individuo de la generación
        mejor_individuo = poblacion[np.argmin(aptitudes)]
        print(f"Generación {generacion + 1}, Mejor costo: {min(aptitudes)}")
        
        # Selección
        poblacion_seleccionada = seleccion(poblacion)
        
        # Cruzamiento y mutación
        nueva_poblacion = []
        for i in range(0, num_individuos, 2):
            if random.random() < probabilidad_cruce:
                hijo1, hijo2 = cruce(poblacion_seleccionada[i], poblacion_seleccionada[i+1])
                nueva_poblacion.append(mutacion(hijo1))
                nueva_poblacion.append(mutacion(hijo2))
            else:
                nueva_poblacion.append(mutacion(poblacion_seleccionada[i]))
                nueva_poblacion.append(mutacion(poblacion_seleccionada[i+1]))
        
        # Actualizar población
        poblacion = np.array(nueva_poblacion)
    
    # Devolver la mejor solución encontrada
    aptitudes = np.array([calcular_aptitud(individuo) for individuo in poblacion])
    mejor_individuo = poblacion[np.argmin(aptitudes)]
    return mejor_individuo.reshape(4, 4)

# Ejecutar el algoritmo genético
mejor_solucion = algoritmo_genetico()

# Mostrar el resultado final
print("Mejor despacho de energía:")
print(mejor_solucion)



KeyboardInterrupt: 